This example notebook demonstrates how a normal processing pipleline works without batch processing. The outline of processing steps are as follows:

1. Clean up the data
2. Perform drift correction
3. Apply light filtering to the data to prepare for merging
4. Merge localizations into one
5. Apply any final filtering

### Import the necessary libraries

In [1]:
%pylab
from bstore import processors as proc
from pathlib import Path
import pandas as pd

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [11]:
# Press TAB inside the quotation marks
filePath = Path('X:/Christian-Sieben/data_HTP/locResults/Cos7_PCSIR_bTub_10ms_PBS_1/Cos7_PCSIR_bTub_10ms_PBS_1_MMStack_Pos0_locResults.dat')

In [12]:
# str() converts the Path to a string
# 'r' means to open the file in read-mode
# df holds the DataFrame returned from pd.read_csv()
with open(str(filePath), 'r') as file:
    df = pd.read_csv(file)

C:\Users\laboleb\Anaconda3\envs\DataSTORM\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
# df.describe()

In [14]:
cleaner = proc.CleanUp()
df      = cleaner(df)

# Save only cleaned data

In [15]:
outputFile = filePath.parent /  Path(filePath.stem + '_cleaned' + filePath.suffix)
print(outputFile)

X:\Christian-Sieben\data_HTP\locResults\Cos7_PCSIR_bTub_10ms_PBS_1\Cos7_PCSIR_bTub_10ms_PBS_1_MMStack_Pos0_locResults_cleaned.dat


In [16]:
with open(str(outputFile), 'w') as file:
    df.to_csv(file)

# Proceed with drift correction

In [ ]:
corrector = proc.FiducialDriftCorrect(minFracFiducialLength = 0.75, # Fiducials must span 75% of number of frames
                                      interactiveSearch     = True, # Select fiducials by eye
                                      noLinking             = True, # Do not perform Crocker-Grier linking
                                      noClustering          = True) # Do not spatially cluster fiducials

When the corrector is run, it will display a 2D histogram image. You may zoom in and out of regions and draw a rectangle around areas with large counts. Areas with counts that are approximately equal to the number of frames are likely to be fiducials. There is a fiducial in this dataset in three bins around (x = 28, y = 55.5).

With the selection rectangle around a region, press `SPACE` to add the region to the list of areas to search for fiducials. Press `r` if you want to reset the regions to empty. When you are done, simply close the window.

If no region is selected, the fiducial search will be performed over the whole set of localizations, which can either be slow or lead to completely wrong results if linking and clustering are turned off.

Also note that the corrector removes fiducials, so it is best to save the output to another DataFrame, in this case `corrDF`.

In [ ]:
corrDF = corrector(df)

We can check the quality of the drift correction curves using `plotFiducials()`.

In [ ]:
corrector.plotFiducials()

It's not necessary for this dataset, but if the drift correction could be improved, we can adjust some of the smoothing parameters and rerun the drift correction. For this example, we'll turn on linking and throw out trajectories shorter than ten consecutive frames. Additionally, we'll shrink the size of the smoothing window and filters to better capture changes in the fiducial trajectory. There are a few parameters for linking and clustering, but we'll leave them at their defaults.

Note that clustering the fiducials can often help get rid of noisy points. However, DBSCAN breaks down if the fiducials are more than about 50,000 frames, so it is preferable to turn it off if you have a long fiducial track as in this example.

In [ ]:
corrector = proc.FiducialDriftCorrect(minFracFiducialLength = 0.75,   # Fiducials must span 75% of number of frames
                                      interactiveSearch     = True,   # Select fiducials bye eye
                                      noLinking             = False,  # Perform Crocker-Grier linking
                                      noClustering          = True,   # Perform DBSCAN to cluster fiducials
                                      smoothingWindowSize   = 750,    # Set the moving window size for smoothing
                                      smoothingFilterSize   = 100)    # Set Gaussian filter std. dev. for smoothing

In [ ]:
# corrDF = corrector(df)

In [ ]:
# corrector.plotFiducials()

We can now investigate the corrected localizations. The x and y columns now contain the corrected localizations. `dx` and `dy` contain the amount of the correction. To get the original data back, one can simply add `dx` to `x` and the same for `y`.

Note that the new count is less than the original one. This is because the drift correction removed localizations belonging to the fiducial marker.

In [ ]:
# corrDF.describe()

# Filtering the data
At this point, we can now filter the data by setting criteria on the columns. First we define the filters. After that, we simply apply them in reverse order to the DataFrame to get the filtered data.

In [ ]:
filter1 = proc.Filter('sigma [nm]', '<', 200)
filter2 = proc.Filter('sigma [nm]', '>', 100)
filter3 = proc.Filter('loglikelihood', '<', 400)

fcDF = filter3(filter2(filter1(corrDF))) # First filter1 is applied, then filter2 is applied.


In [ ]:
# fcDF.describe()

# Save only drift corrected

In [ ]:
outputFile = filePath.parent /  Path(filePath.stem + '_DC' + filePath.suffix)
print(outputFile)

In [ ]:
with open(str(outputFile), 'w') as file:
    fcDF.to_csv(file)

# Merging localizations
The last step in the analysis pipeline typically involves merging localizations that are on for several frames into one. This is performed by the Crocker-Grier algorithm in trackpy, but all you have to worry about it defining a Merge processor and applying it to the DataFrame.


In [ ]:
import importlib
importlib.reload(proc)

In [ ]:
merger = proc.Merge(tOff            = 1,  # Number of frames that a molecule can be missing and still be part of a track
                    mergeRadius     = 40) # Maximum distance between successive molecules

mfcDF = merger(fcDF)

In [ ]:
mfcDF.describe()

Final filtering and saving
At this point, the data may be filtered once more in the same manner as above. Let's skip this part and save the data to disk.

In [ ]:
outputFile = filePath.parent /  Path(filePath.stem + '_DC_Merged' + filePath.suffix)
print(outputFile)

In [ ]:
with open(str(outputFile), 'w') as file:
    mfcDF.to_csv(file)

Save only drift corrected